In [1]:
# импортируем библиотеки numpy и pandas
import numpy as np
import pandas as pd

In [2]:
# загружаем данные
data = pd.read_csv('Data/example.csv', sep='\t')
data

,date,sales
0,09.01.2018,2400
1,10.01.2018,2800
2,11.01.2018,2500
3,12.01.2018,2890
4,13.01.2018,2610
5,14.01.2018,2500
6,15.01.2018,2750
7,16.01.2018,2700
8,17.01.2018,2250
9,18.01.2018,2350


In [3]:
# преобразовываем столбец с датой в тип datetime
data['date'] = pd.to_datetime(data['date'], format='%d.%m.%Y')
# создаем лаги с запаздыванием на 3, 5 и 8 дней
data['Lag3'] = data['sales'].shift(3)
data['Lag4'] = data['sales'].shift(4)
data['Lag5'] = data['sales'].shift(5)
# смотрим результаты
data

,date,sales,Lag3,Lag4,Lag5
0,2018-01-09,2400,NaN,NaN,NaN
1,2018-01-10,2800,NaN,NaN,NaN
2,2018-01-11,2500,NaN,NaN,NaN
3,2018-01-12,2890,2400.0,NaN,NaN
4,2018-01-13,2610,2800.0,2400.0,NaN
5,2018-01-14,2500,2500.0,2800.0,2400.0
6,2018-01-15,2750,2890.0,2500.0,2800.0
7,2018-01-16,2700,2610.0,2890.0,2500.0
8,2018-01-17,2250,2500.0,2610.0,2890.0
9,2018-01-18,2350,2750.0,2500.0,2610.0


In [4]:
# разбиваем набор на обучающую и тестовую выборки
train, test = data[0:len(data)-4], data[len(data)-4:]

In [5]:
# смотрим обучающий набор
train

,date,sales,Lag3,Lag4,Lag5
0,2018-01-09,2400,NaN,NaN,NaN
1,2018-01-10,2800,NaN,NaN,NaN
2,2018-01-11,2500,NaN,NaN,NaN
3,2018-01-12,2890,2400.0,NaN,NaN
4,2018-01-13,2610,2800.0,2400.0,NaN
5,2018-01-14,2500,2500.0,2800.0,2400.0
6,2018-01-15,2750,2890.0,2500.0,2800.0
7,2018-01-16,2700,2610.0,2890.0,2500.0


In [6]:
# смотрим тестовый набор
test

,date,sales,Lag3,Lag4,Lag5
8,2018-01-17,2250,2500.0,2610.0,2890.0
9,2018-01-18,2350,2750.0,2500.0,2610.0
10,2018-01-19,2550,2700.0,2750.0,2500.0
11,2018-01-20,3000,2250.0,2700.0,2750.0


In [7]:
# пишем функцию для создания лагов на тесте
def lags_create_on_test(train, test, target, horizon, lags_range):
    """
    Создает лаги в тесте
    
    Параметры
    ----------
    train: обучающий набор
    test: тестовый набор
    target: название зависимой переменной
    horizon: горизонт прогнозирования
    lags_range: диапазон значений порядка лагов
    """
    df = pd.concat([train, test], axis=0)
    for i in lags_range:
        df['Lag_' + f'{i}'] = df[target].shift(i)
    test = df.tail(horizon)
    return test

In [8]:
# создаем лаги в тесте
tst = lags_create_on_test(train, test, 4, range(4, 6))
tst

,date,sales,Lag3,Lag4,Lag5,Lag_4,Lag_5
8,2018-01-17,2250,2500.0,2610.0,2890.0,2610.0,2890.0
9,2018-01-18,2350,2750.0,2500.0,2610.0,2500.0,2610.0
10,2018-01-19,2550,2700.0,2750.0,2500.0,2750.0,2500.0
11,2018-01-20,3000,2250.0,2700.0,2750.0,2700.0,2750.0


In [9]:
# загружаем данные
data = pd.read_csv('Data/example.csv', sep='\t')
# создаем скользящее среднее, окно шириной 3
data['rolling_mean3'] = data['sales'].rolling(window=3).mean()
data

,date,sales,rolling_mean3
0,09.01.2018,2400,NaN
1,10.01.2018,2800,NaN
2,11.01.2018,2500,2566.666667
3,12.01.2018,2890,2730.000000
4,13.01.2018,2610,2666.666667
5,14.01.2018,2500,2666.666667
6,15.01.2018,2750,2620.000000
7,16.01.2018,2700,2650.000000
8,17.01.2018,2250,2566.666667
9,18.01.2018,2350,2433.333333


In [10]:
# создаем скользящее среднее, окно шириной 3 и минимальным
# количеством наблюдений в окне, равным 2
data['rolling_mean3_min_periods_2'] = data['sales'].rolling(
    window=3, min_periods=2).mean()
data

,date,sales,rolling_mean3,rolling_mean3_min_periods_2
0,09.01.2018,2400,NaN,NaN
1,10.01.2018,2800,NaN,2600.000000
2,11.01.2018,2500,2566.666667,2566.666667
3,12.01.2018,2890,2730.000000,2730.000000
4,13.01.2018,2610,2666.666667,2666.666667
5,14.01.2018,2500,2666.666667,2666.666667
6,15.01.2018,2750,2620.000000,2620.000000
7,16.01.2018,2700,2650.000000,2650.000000
8,17.01.2018,2250,2566.666667,2566.666667
9,18.01.2018,2350,2433.333333,2433.333333


In [11]:
# удалим прежние скользящие средние
data.drop(['rolling_mean3', 'rolling_mean3_min_periods_2'], 
          axis=1, inplace=True)
# разбиваем набор на обучающую и тестовую выборки
train, test = data[0:len(data)-4], data[len(data)-4:]

# отключим предупреждения
import warnings
warnings.filterwarnings('ignore')

In [12]:
# считаем скользящее среднее, окно шириной 4, 
# c лагом 1, на обучающей выборке
train['rolling_mean4'] = train['sales'].shift(1).rolling(
    window=4).mean()
train

,date,sales,rolling_mean4
0,09.01.2018,2400,NaN
1,10.01.2018,2800,NaN
2,11.01.2018,2500,NaN
3,12.01.2018,2890,NaN
4,13.01.2018,2610,2647.5
5,14.01.2018,2500,2700.0
6,15.01.2018,2750,2625.0
7,16.01.2018,2700,2687.5


In [13]:
# вручную считаем скользящее среднее, окно шириной 4, 
# c лагом 1, на обучающей выборке
train['rolling_mean4_manually'] = np.NaN
train.iloc[4, 3] = (2400 + 2800 + 2500 + 2890) / 4
train.iloc[5, 3] = (2800 + 2500 + 2890 + 2610) / 4
train.iloc[6, 3] = (2500 + 2890 + 2610 + 2500) / 4
train.iloc[7, 3] = (2890 + 2610 + 2500 + 2750) / 4
train

,date,sales,rolling_mean4,rolling_mean4_manually
0,09.01.2018,2400,NaN,NaN
1,10.01.2018,2800,NaN,NaN
2,11.01.2018,2500,NaN,NaN
3,12.01.2018,2890,NaN,NaN
4,13.01.2018,2610,2647.5,2647.5
5,14.01.2018,2500,2700.0,2700.0
6,15.01.2018,2750,2625.0,2625.0
7,16.01.2018,2700,2687.5,2687.5


In [14]:
# создаем в тестовой выборке столбец rolling_mean4, 
# состоящий из значений NaN
test['rolling_mean4'] = np.NaN
test

,date,sales,rolling_mean4
8,17.01.2018,2250,NaN
9,18.01.2018,2350,NaN
10,19.01.2018,2550,NaN
11,20.01.2018,3000,NaN


In [15]:
# вычисляем первое скользящее среднее в тестовой выборке
test.iloc[0, 2] = (2610 + 2500 + 2750 + 2700) / 4
test

,date,sales,rolling_mean4
8,17.01.2018,2250,2640.0
9,18.01.2018,2350,NaN
10,19.01.2018,2550,NaN
11,20.01.2018,3000,NaN


In [16]:
# вычисляем остальные скользящие средние в тестовой выборке
test.iloc[1, 2] = (2500 + 2750 + 2700) / 3
test.iloc[2, 2] = (2750 + 2700) / 2
test.iloc[3, 2] = 2700
test

,date,sales,rolling_mean4
8,17.01.2018,2250,2640.0
9,18.01.2018,2350,2650.0
10,19.01.2018,2550,2725.0
11,20.01.2018,3000,2700.0
